<a href="https://colab.research.google.com/github/kolayn808/textattack_test/blob/main/Seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установка необходимых библиотек

In [8]:
!pip install transformers
!pip install nltk

Загрузка моделей и стоп-слов.  

In [34]:
import random
import nltk
import gensim.downloader
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from IPython.core.display import display, HTML


# Загрузка модели Word2Vec
word2vec_model = gensim.downloader.load("glove-wiki-gigaword-50")

# Загрузка токенизатора и модели seq2seq
tokenizer = AutoTokenizer.from_pretrained("t5-small")  # Замените на желаемую модель seq2seq
my_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")  # Замените на желаемую модель seq2seq

# Загрузка стоп-слов
nltk.download('punkt')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Функция атаки

In [35]:

def attack_sequence_to_sequence_model(input_text, model, max_attempts=10, max_edits=5):
    original_text = input_text
    for _ in range(max_attempts):
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=128, truncation=True, padding=True)
        num_edits = random.randint(1, max_edits)
        for _ in range(num_edits):
            input_word = random.choice(input_ids[0].tolist())
            input_word = tokenizer.convert_ids_to_tokens([input_word])[0]
            if input_word not in stop_words and input_word in word2vec_model:
                similar_words = word2vec_model.most_similar(input_word, topn=5)
                new_word = random.choice(similar_words)[0]
                input_text = input_text.replace(input_word, new_word, 1)

        # Генерируем новый текст с моделью seq2seq
        generated_ids = my_model.generate(input_ids, max_length=128, num_return_sequences=1)
        generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        if generated_text != original_text:
            return generated_text  # Возвращаем атакованный текст
    return None  # Возвращаем None, если атака не удалась

Пример использования

In [36]:

original_text = "Two men wearing swim trunks jump in the air at a moderately populated beach"
attacked_text = attack_sequence_to_sequence_model(original_text, my_model)
if attacked_text:
    print("Оригинальный текст:", original_text)
    print("Атакованный текст:", attacked_text)
else:
    print("Атака не удалась.")

Оригинальный текст: Two men wearing swim trunks jump in the air at a moderately populated beach
Атакованный текст: Zwei Männer in swim trunk jump in the air at a moderately populated beach.
